<a href="https://colab.research.google.com/github/olive-su/AI_codepresso/blob/master/DataAugmentation%EC%9D%84_%ED%86%B5%ED%95%9C_%EA%B0%80%EC%9C%84_%EB%B0%94%EC%9C%84_%EB%B3%B4_%EB%B6%84%EB%A5%98_%EB%B6%84%EC%84%9D_%EB%AA%A8%EB%8D%B8%EC%9D%98_%EC%84%B1%EB%8A%A5_%EA%B0%9C%EC%84%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow import keras

## Data Augmention 을 통한 가위-바위-보 데이터 셋 분류 성능 개선

### Step 1. Input tensor 와 Target tensor 준비(훈련데이터)

(1) 가위-바위-보 데이터셋 다운로드

In [3]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip'

path_to_zip = keras.utils.get_file('rps.zip',
                                   origin=url,
                                   extract=True,
                                   cache_dir='/content')

200695808/200682221 [==============================] - 5s 0us/step


In [4]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip'

path_to_zip = keras.utils.get_file('rps_test.zip',
                                   origin=url,
                                   extract=True,
                                   cache_dir='/content')

29532160/29516758 [==============================] - 1s 0us/step


(2) ImageDataGenerator를 이용해 이미지 파일을 load 하기 위한 경로 지정

In [5]:
train_dir = '/content/datasets/rps'
test_dir = '/content/datasets/rps-test-set'

(3) ImageDataGenerator 객체 생성  
* 객체 생성 시 rescale 인자를 이용하여 텐서 내 원소의 범위를 [0 ~ 255] => [0 ~ 1] 로 ReScaling 진행
* Data Augmentation 을 위한 정보들 설정

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Augmentation 수행 인자
# rotation : 40
# width : 0.2
# hight : 0.2
# share : 0.2
# zoom : 0.2
# horizontal : True
train_datagen = ImageDataGenerator(
    rescale=1./225.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255.)

* .flow_from_directory() 메서드를 이용하여 학습데이터와 검증데이터를 위한 DirectoryIterator 객체 생성

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    shuffle=True,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 2520 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


### Step 2. CNN 모델 디자인 및 학습 정보 설정

(1) CNN 모델 구성

In [9]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(3, activation='sigmoid'))

(2) 학습을 위한 설정 정보 지정

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Step 3. 모델에 데이터 generator 연결 후 학습 
  * model.fit() 이용하여 데이터 연결 및 학습시키기

In [11]:
history = model.fit(
      train_generator,
      steps_per_epoch=len(train_generator),
      epochs=10,
      validation_data=validation_generator,
      validation_steps=len(validation_generator))

Epoch 1/10
79/79 [==============================] - 37s 313ms/step - loss: 1.2281 - accuracy: 0.4179 - val_loss: 0.8705 - val_accuracy: 0.4802
Epoch 2/10
79/79 [==============================] - 24s 306ms/step - loss: 0.8446 - accuracy: 0.6333 - val_loss: 1.8708 - val_accuracy: 0.4385
Epoch 3/10
79/79 [==============================] - 25s 310ms/step - loss: 0.5233 - accuracy: 0.7861 - val_loss: 0.1804 - val_accuracy: 0.9722
Epoch 4/10
79/79 [==============================] - 25s 311ms/step - loss: 0.2997 - accuracy: 0.8877 - val_loss: 0.0554 - val_accuracy: 1.0000
Epoch 5/10
79/79 [==============================] - 25s 310ms/step - loss: 0.2170 - accuracy: 0.9167 - val_loss: 0.0733 - val_accuracy: 0.9683
Epoch 6/10
79/79 [==============================] - 24s 309ms/step - loss: 0.1899 - accuracy: 0.9381 - val_loss: 0.0972 - val_accuracy: 0.9623
Epoch 7/10
79/79 [==============================] - 24s 309ms/step - loss: 0.1370 - accuracy: 0.9492 - val_loss: 0.5923 - val_accuracy: 0.6944

### Step 4. 테스트 데이터 셋을 통한 모델의 성능 평가

In [12]:
loss, accuracy = model.evaluate(test_generator)

12/12 [==============================] - 2s 143ms/step - loss: 0.0870 - accuracy: 0.9597
